In [1]:
import os

In [2]:
import librosa

In [3]:
import mir_eval

In [4]:
from collections import OrderedDict

In [5]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=3)
pd.set_option('precision', 4, "display.max_rows", 999)

In [6]:
def make_onset_corpus(onset_path):
    
    # Beat files
    audio = librosa.util.find_files(onset_path, ext='wav')
    
    annotations = [af.replace('.wav', '.onsets') for af in audio]
    
    data = []
    for aud, ann in zip(audio, annotations):
        if os.path.exists(aud) and os.path.exists(ann):
            data.append((aud, ann))
    
    return pd.DataFrame(data=data, columns=['audio', 'annotation'])

In [7]:
def make_output_path(base, outpath):
    
    root = os.path.splitext(base)[0]
    
    output = os.path.join(outpath, os.path.extsep.join([root, 'json']))
    
    return output

In [27]:
def analyze(dframe, outpath='/home/bmcfee/git/librosa_parameters/data/onset'):
    
    index = dframe.index[0]
    base = os.path.basename(dframe['audio'][index])
    
    outfile = make_output_path(base, outpath)
    
    if os.path.exists(outfile):
        print 'Cached {}'.format(base)
        data = pd.read_json(outfile, orient='records')
        return data
    else:
        print 'Processing {}'.format(base)
    
    # Load the truth
    ref_times = np.sort(pd.read_table(dframe['annotation'][index], header=None, sep='\s+')[0].values)

    # Load the audio
    y, sr = librosa.load(dframe['audio'][index])
    
    # Construct the output container
    results = []
    hop_length = 512
    
    effective_sr = sr // hop_length
    
    # Onset strength parameters
    for fmax in [8000, 11025]:
        for n_mels in [32, 64, 128]:
            S = librosa.feature.melspectrogram(y=y, hop_length=hop_length, sr=sr, fmax=fmax, n_mels=n_mels)
            S = librosa.logamplitude(S)
            
            for aggregate in [np.mean, np.median]:
        
                # Compute the onset detection function
                oenv = librosa.onset.onset_strength(S=S,
                                                    sr=sr,
                                                    hop_length=hop_length,
                                                    aggregate=aggregate)
                
                for delta in [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]:
                    onsets = librosa.onset.onset_detect(onset_envelope=oenv, sr=sr,
                                                        delta=delta, hop_length=hop_length)
                    
                    params = {'aggregate': aggregate.__name__,
                              'fmax': fmax,
                              'n_mels': n_mels,
                              'delta': delta,}
                             
                    est_times = librosa.frames_to_time(onsets, sr=sr, hop_length=hop_length)
                    scores = mir_eval.onset.evaluate(ref_times, est_times)
                            
                    cont = OrderedDict(index=index)
                    cont.update(params)
                    cont.update(scores)
                    results.append(cont)
                    
    # Blow away the cache
    #librosa.cache.clear()
    data = pd.DataFrame.from_dict(results, orient='columns')
    data.to_json(outfile, orient='records')
        
    return data

In [28]:
def analyze_corpus(corpus):
    
    results = None
    for idx in corpus.index:
        new_results = analyze(corpus.loc[[idx]])
        if results is None:
            results = new_results
        else:
            results = pd.concat([results, new_results])
            
    return results

In [29]:
from joblib import Parallel, delayed

In [30]:
def p_analyze_corpus(corpus, n_jobs=3):
    
    results = None
    
    dfunc = delayed(analyze)
    
    results = Parallel(n_jobs=n_jobs, verbose=10)(dfunc(corpus.loc[[idx]])
                                                  for idx in corpus.index)

    return pd.concat(results)

---

In [31]:
onset_data = make_onset_corpus('/home/bmcfee/data/onsets/clean_data/')

In [32]:
onset_results = p_analyze_corpus(onset_data)

[Parallel(n_jobs=3)]: Done   1 out of 351 | elapsed:    1.5s remaining:  8.7min
[Parallel(n_jobs=3)]: Done  29 out of 351 | elapsed:   10.7s remaining:  2.0min
[Parallel(n_jobs=3)]: Done  65 out of 351 | elapsed:   24.5s remaining:  1.8min
[Parallel(n_jobs=3)]: Done 101 out of 351 | elapsed:   39.4s remaining:  1.6min
[Parallel(n_jobs=3)]: Done 137 out of 351 | elapsed:   45.2s remaining:  1.2min
[Parallel(n_jobs=3)]: Done 173 out of 351 | elapsed:   53.8s remaining:   55.4s
[Parallel(n_jobs=3)]: Done 209 out of 351 | elapsed:  1.0min remaining:   42.2s
[Parallel(n_jobs=3)]: Done 245 out of 351 | elapsed:  1.1min remaining:   29.6s
[Parallel(n_jobs=3)]: Done 281 out of 351 | elapsed:  1.4min remaining:   20.3s
[Parallel(n_jobs=3)]: Done 317 out of 351 | elapsed:  1.6min remaining:   10.0s
[Parallel(n_jobs=3)]: Done 351 out of 351 | elapsed:  1.7min finished
/home/bmcfee/git/mir_eval/mir_eval/onset.py:51: UserWarning: Estimated onsets are empty.
  warnings.warn("Estimated onsets are emp

Processing ah_cello_03-Cello_Sonata_3__I_Allegro_ma_non_tanto_pt1.wav
Processing ah_cello_03_CelloTaksimi_pt1.wav
Processing ah_cello_08_-_Bach_(JS)-_Cello_Suite_-4_In_E_Flat_BWV_1010i_-_1._Preludium.wav
Processing ah_clarinet_44361_debudding_Clarinet_ORTF_Stereo_Pair_NT_5_s_01.wavProcessing ah_cello_cello1.wavProcessing ah_cello_14_VioloncelloTaksim_pt1.wav


Processing ah_clarinet_clarinet1.wavProcessing ah_clarinet_SL1_pt1.wavProcessing ah_clarinet_44784_alikirodgers_AB_Clarinet_01_pt1.wav


Processing ah_guitar_Guitar_Licks_06-12.wavProcessing ah_guitar_2684_TexasMusicForge_Dandelion_pt1.wavProcessing ah_clarinet_my_clarinet1.wav


Processing ah_guitar_Guitar_Licks_15-05.wavProcessing ah_guitar_Guitar_Licks_07-06.wavProcessing ah_guitar_Guitar_Licks_06-11.wav


Processing ah_guitar_Summer_Together_110_pt1.wavProcessing ah_guitar_Guitar_Licks_15-06.wavProcessing ah_guitar_Guitar_Licks_07-11.wav


Processing ah_guitar_my_guitar1.wavProcessing ah_guitar_guitar2.wavProcessing ah_guitar

In [33]:
onset_results.to_json('/home/bmcfee/git/librosa_parameters/onset_results.json', orient='records')


Processing jpb_2_T46-tenor.wavProcessing jpb_2_Old Swinger 3.wavProcessing jpb_2_Slow burner 05.wav


Processing jpb_2_Talking Drum 03.wavProcessing jpb_2_Reggaedrum_080_loop02.wavProcessing jpb_2_T08-violin.wav


Processing jpb_2_arab60s.wavProcessing jpb_2_Reggaedrum_080_loop10.wavProcessing jpb_2_T26-claves.wav


Processing jpb_2_fjf306.wavProcessing jpb_2_SG047_120_6_8_Groove_05.wavProcessing jpb_2_T37-vibraphone.wav


Processing jpb_2_fourkick7.wavProcessing jpb_2_T08-violin-arp.wavProcessing jpb_2_T41-celesta.wav


Processing jpb_2_jaxx.wavProcessing jpb_2_T11-doublebass.wavProcessing jpb_2_T45-alto.wav


Processing jpb_2_mambo01.wavProcessing jpb_2_T35-glockenspiel.wavProcessing jpb_2_T47-bass.wav


Processing jpb_2_mambo03.wavProcessing jpb_2_T44-soprano.wavProcessing jpb_2_Talking Drum 01.wav


Processing jpb_2_odd step11.wavProcessing jpb_2_T62-soprano-spiritual.wavProcessing jpb_2_Udu 01.wav


Processing jpb_2_queensBeat1_79bpm.wavProcessing jpb_2_cello1.wavProcessing jpb_2

In [34]:
onset_results = pd.read_json('/home/bmcfee/git/librosa_parameters/onset_results.json', orient='records')

In [35]:
onset_scores = onset_results.groupby(['aggregate', 'fmax', 'n_mels', 'delta']).mean()

In [36]:
onset_scores['F-measure'].argmax()

(u'mean', 11025, 128, 0.070000000000000007)

In [37]:
best_f = onset_scores['F-measure'].argmax()

In [38]:
onset_scores.loc[best_f]

F-measure      0.749
Precision      0.782
Recall         0.770
index        175.000
Name: (mean, 11025, 128, 0.07), dtype: float64

In [39]:
onset_scores

F-measure  Precision  Recall  index
aggregate fmax  n_mels delta                                     
mean      8000  32     0.00       0.526      0.447   0.833    175
                       0.01       0.641      0.598   0.818    175
                       0.02       0.681      0.661   0.798    175
                       0.03       0.702      0.700   0.781    175
                       0.04       0.709      0.723   0.761    175
                       0.05       0.711      0.740   0.741    175
                       0.06       0.709      0.752   0.722    175
                       0.07       0.706      0.762   0.704    175
                       0.08       0.699      0.773   0.683    175
                       0.09       0.689      0.778   0.662    175
                       0.10       0.685      0.788   0.649    175
                64     0.00       0.527      0.444   0.847    175
                       0.01       0.647      0.598   0.835    175
                       0.02       0.692      0.667   0.821    175
                       0.03       0.716      0.708   0.805    175
                       0.04       0.726      0.734   0.788    175
                       0.05       0.732      0.754   0.775    175
                       0.06       0.732      0.765   0.758    175
                       0.07       0.731      0.779   0.742    175
                       0.08       0.727      0.787   0.725    175
                       0.09       0.724      0.796   0.712    175
                       0.10       0.719      0.806   0.695    175
                128    0.00       0.526      0.441   0.854    175
                       0.01       0.645      0.591   0.844    175
                       0.02       0.691      0.661   0.832    175
                       0.03       0.717      0.701   0.820    175
                       0.04       0.732      0.730   0.808    175
                       0.05       0.739      0.751   0.795    175
                       0.06       0.741      0.765   0.781    175
                       0.07       0.748      0.785   0.768    175
                       0.08       0.746      0.794   0.753    175
                       0.09       0.743      0.804   0.738    175
                       0.10       0.739      0.814   0.724    175
          11025 32     0.00       0.529      0.450   0.832    175
                       0.01       0.645      0.603   0.816    175
                       0.02       0.685      0.664   0.798    175
                       0.03       0.704      0.702   0.778    175
                       0.04       0.709      0.722   0.760    175
                       0.05       0.712      0.739   0.740    175
                       0.06       0.708      0.749   0.721    175
                       0.07       0.703      0.756   0.702    175
                       0.08       0.697      0.764   0.684    175
                       0.09       0.693      0.772   0.669    175
                       0.10       0.687      0.778   0.654    175
                64     0.00       0.530      0.447   0.848    175
                       0.01       0.650      0.603   0.835    175
                       0.02       0.696      0.672   0.822    175
                       0.03       0.719      0.709   0.808    175
                       0.04       0.726      0.731   0.790    175
                       0.05       0.732      0.748   0.776    175
                       0.06       0.732      0.762   0.759    175
                       0.07       0.730      0.772   0.741    175
                       0.08       0.727      0.782   0.726    175
                       0.09       0.722      0.790   0.710    175
                       0.10       0.717      0.798   0.695    175
                128    0.00       0.529      0.445   0.856    175
                       0.01       0.650      0.598   0.845    175
                       0.02       0.697      0.667   0.836    175
                       0.03       0.718      0.701   0.822    175
    

In [62]:
# Previous configuration
onset_scores.loc[(u'mean', 11025, 128, 0.05)]

F-measure      0.741
Precision      0.751
Recall         0.796
index        175.000
Name: (mean, 11025, 128, 0.05), dtype: float64

In [64]:
best_f

(u'mean', 11025, 128, 0.070000000000000007)

In [69]:
onset_scores.loc[(u'mean', 8000, 128, 0.07)]

F-measure      0.748
Precision      0.785
Recall         0.768
index        175.000
Name: (mean, 8000, 128, 0.07), dtype: float64